In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.fft import fft, fftshift
from scipy.special import jv, yv # Bessel functions J_n and Y_n
from scipy.interpolate import lagrange # Lagrange polynomials

# --- 常數與參數設定 ---
Phi0 = 1.0       # 磁通量量子 (歸一化)
W = 1.0          # 接面寬度 (歸一化)
N_points_x = 256 # 接面空間取樣點數
x_coords = np.linspace(-W/2, W/2, N_points_x) # 接面寬度座標

# 掃描的外部磁通量範圍 (歸一化)
phi_ext_over_phi0_scan = np.linspace(-8, 8, 800) # 擴大範圍以觀察更多旁瓣

# --- 定義不同的電流密度分佈函數 Jc(x) ---
def uniform_Jc(x_coords, W_val):
    """均勻電流密度"""
    return np.ones_like(x_coords)

def symmetric_edge_gaussian_Jc(x_coords, W_val, peak_width_factor=0.1, edge_offset_factor=0.05):
    """對稱邊緣集中的電流密度 (兩個高斯峰)"""
    sigma = W_val * peak_width_factor
    peak1_center = -W_val/2 + W_val * edge_offset_factor + sigma 
    peak2_center = W_val/2 - W_val * edge_offset_factor - sigma
    return np.exp(-(x_coords - peak1_center)**2 / (2 * sigma**2)) + \
           np.exp(-(x_coords - peak2_center)**2 / (2 * sigma**2))

def asymmetric_linear_Jc(x_coords, W_val):
    """不對稱線性梯度電流密度 (僅正值)"""
    return 0.7 + 0.5 * (x_coords / (W_val/2)) 

def one_side_step_Jc(x_coords, W_val, step_start_norm=-0.5, step_end_norm=-0.2):
    """只有一邊的不對稱邊緣集中 (階梯)"""
    J = np.zeros_like(x_coords)
    start_x = step_start_norm * W_val
    end_x = step_end_norm * W_val
    actual_start = np.maximum(start_x, -W_val/2)
    actual_end = np.minimum(end_x, W_val/2)
    if actual_start < actual_end:
         J[(x_coords >= actual_start) & (x_coords <= actual_end)] = 1.0
    return J

def asymmetric_multistep_Jc(x_coords, W_val, x_split_norm=-0.1, h1=1.0, h2=0.4):
    """不對稱多階梯 (兩段不同高度, 僅正值)"""
    J = np.zeros_like(x_coords)
    x_split = x_split_norm * W_val
    J[x_coords <= x_split] = h1
    J[x_coords > x_split] = h2
    J[x_coords < -W_val/2] = 0
    J[x_coords > W_val/2] = 0
    return J

def symmetric_edge_steps_Jc(x_coords, W_val, depth_norm=0.2, base_level=0.0):
    """雙邊均勻邊緣集中 (對稱階梯)"""
    J = np.full_like(x_coords, base_level)
    edge_width = depth_norm * W_val
    J[(x_coords >= -W_val/2) & (x_coords <= -W_val/2 + edge_width)] = 1.0
    J[(x_coords >= W_val/2 - edge_width) & (x_coords <= W_val/2)] = 1.0
    return J

def bilateral_opposite_shape_edge_Jc(x_coords, W_val, 
                                     left_step_end_norm=-0.25, left_h=1.0,
                                     right_peak_center_norm=0.3, right_peak_h=0.8, right_peak_sigma_norm=0.07):
    """雙邊異形邊緣集中 (左階梯, 右高斯)"""
    J_left = np.zeros_like(x_coords)
    left_step_end = left_step_end_norm * W_val
    J_left[(x_coords >= -W_val/2) & (x_coords <= left_step_end)] = left_h 

    J_right = np.zeros_like(x_coords)
    right_peak_center = right_peak_center_norm * W_val
    right_sigma = right_peak_sigma_norm * W_val
    J_right = right_peak_h * np.exp(-(x_coords - right_peak_center)**2 / (2 * right_sigma**2)) 

    J_combined = np.zeros_like(x_coords)
    mask_left = (x_coords >= -W_val/2) & (x_coords <= left_step_end)
    mask_right_exclusive = (x_coords > left_step_end) 

    J_combined[mask_left] = J_left[mask_left]
    J_combined[mask_right_exclusive] = J_right[mask_right_exclusive]
    return J_combined

def Jc_with_linear_phase_gradient(x_coords, W_val, J_amplitude_func, phase_k0_norm):
    """帶有內稟線性相位梯度的電流密度 (用於產生位移的對稱圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    k_intrinsic = phase_k0_norm * np.pi 
    return J_real * np.exp(1j * k_intrinsic * (x_coords / (W/2)))

def asymmetric_phi0_profile(x_coords, W_val, A=np.pi/2, x_d_norm=-0.2, sigma_norm=0.15):
    """一個不對稱的內稟相位移 phi_0^{int}(x) 範例 (例如，一側的相位突變)"""
    x_d = x_d_norm * W_val
    sigma = sigma_norm * W_val
    return A * np.exp(-(x_coords - x_d)**2 / (2 * sigma**2))

def Jc_with_asymmetric_phi0(x_coords, W_val, J_amplitude_func, phi0_int_func, phi0_params):
    """帶有不對稱內稟相位移的電流密度 (用於產生不對稱的Ic(Phi)圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    phi0_intrinsic = phi0_int_func(x_coords, W_val, **phi0_params)
    return J_real * np.exp(-1j * phi0_intrinsic) # J_eff = J_c * exp(-i * phi_0_int)

# 選定的電流密度分佈與數學表達式
current_distributions = {
    "均勻 (Uniform)": {
        "profile": uniform_Jc(x_coords, W),
        "math": r"$J_c(x) = J_0$ (constant)"
    },
    "對稱邊緣高斯 (Symm. Edge Gaussian)": {
        "profile": symmetric_edge_gaussian_Jc(x_coords, W, peak_width_factor=0.08, edge_offset_factor=0.05),
        "math": r"$J_c(x) = \exp\left(-\frac{(x-x_1)^2}{2\sigma^2}\right) + \exp\left(-\frac{(x-x_2)^2}{2\sigma^2}\right)$"
    },
    "不對稱線性 (僅正值) (Asymm. Linear)": {
        "profile": asymmetric_linear_Jc(x_coords, W),
        "math": r"$J_c(x) = 0.7 + 0.5 \cdot \frac{x}{W/2}$"
    },
    "單邊階梯 (One-side Step)": {
        "profile": one_side_step_Jc(x_coords, W, step_start_norm=-0.5, step_end_norm=-0.2),
        "math": r"$J_c(x) = \begin{cases} J_0 & \text{if } x_1 \leq x \leq x_2 \\ 0 & \text{otherwise} \end{cases}$"
    },
    "不對稱多階梯 (僅正值) (Asymm. Multi-Steps)": {
        "profile": asymmetric_multistep_Jc(x_coords, W, x_split_norm=-0.1, h1=1.0, h2=0.4),
        "math": r"$J_c(x) = \begin{cases} h_1 & \text{if } x \leq x_s \\ h_2 & \text{if } x > x_s \end{cases}$"
    },
    "雙邊均勻邊緣 (Symm. Edge Steps)": {
        "profile": symmetric_edge_steps_Jc(x_coords, W, depth_norm=0.15, base_level=0.0),
        "math": r"$J_c(x) = \begin{cases} J_0 & \text{if } |x| > W/2 - d \\ 0 & \text{otherwise} \end{cases}$"
    },
    "雙邊異形邊緣 (Bilateral Diff. Shapes)": {
        "profile": bilateral_opposite_shape_edge_Jc(x_coords, W, left_step_end_norm=-0.2, left_h=0.9, right_peak_center_norm=0.3, right_peak_h=1.0, right_peak_sigma_norm=0.1),
        "math": r"$J_c(x) = \begin{cases} h_L & \text{left step} \\ h_R \exp\left(-\frac{(x-x_R)^2}{2\sigma_R^2}\right) & \text{right Gaussian} \end{cases}$"
    },
    "均勻Jc+線性相位梯度 (僅正值) (Shifted Uniform)": {
        "profile": Jc_with_linear_phase_gradient(x_coords, W, lambda x, w: uniform_Jc(x,w), phase_k0_norm=0.5),
        "math": r"$J_{eff}(x) = J_0 \exp(i k_0 x)$"
    },
    "均勻Jc+不對稱相位 (僅正值) (Asymmetric Ic(Phi))": {
        "profile": Jc_with_asymmetric_phi0(x_coords, W, 
                                              lambda x, w: uniform_Jc(x,w), # 基底 Jc(x)
                                              asymmetric_phi0_profile,     # 不對稱 phi_0^{int}(x) 函數
                                              {'A': np.pi*0.8, 'x_d_norm': -0.25, 'sigma_norm': 0.2}), # phi_0^{int}(x) 的參數
        "math": r"$J_{eff}(x) = J_0 \exp(-i \phi_0^{int}(x))$, where $\phi_0^{int}(x) = A \exp\left(-\frac{(x-x_d)^2}{2\sigma^2}\right)$"
    }
}

# 歸一化電流密度分佈
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    if np.iscomplexobj(Jc_profile_complex):
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9:
            current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val
    else: 
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9: 
             current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val 

# 計算 Ic(Phi) 圖樣
Ic_patterns = {}
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    Ic_vs_phi = []
    for phi_norm in phi_ext_over_phi0_scan:
        integrand = Jc_profile_complex * np.exp(1j * 2 * np.pi * phi_norm * (x_coords / W))
        dx = W / (N_points_x -1) if N_points_x > 1 else W 
        Ic_val = np.abs(np.sum(integrand) * dx) 
        Ic_vs_phi.append(Ic_val)

    Ic_patterns[name] = np.array(Ic_vs_phi)
    max_ic_val_pattern = np.max(Ic_patterns[name])
    if max_ic_val_pattern > 1e-9:
         Ic_patterns[name] /= max_ic_val_pattern

# 創建圖表
fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=False,
                    row_heights=[0.35, 0.35, 0.3],
                    subplot_titles=("電流密度分佈 $J_{eff}(x)$ (實部與虛部，已歸一化至$|\cdot|_{max}=1$)", 
                                    "對應的夫琅禾費圖樣 $I_c(\Phi_{ext})$ (已歸一化至$I_{c,peak}=1$)",
                                    "數學表達式"))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# 繪製電流密度分佈
for i, (name, data) in enumerate(current_distributions.items()):
    Jc_profile_complex = data["profile"]
    color = colors[i % len(colors)]
    
    if np.iscomplexobj(Jc_profile_complex):
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.real(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (實部)', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.imag(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (虛部)', legendgroup=name, 
                                 line=dict(color=color, dash='dot')),
                      row=1, col=1)
    else:
        fig.add_trace(go.Scatter(x=x_coords/W, y=Jc_profile_complex, mode='lines', 
                                 name=f'{name}', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)

    # 繪製 Ic(Phi) 圖樣
    fig.add_trace(go.Scatter(x=phi_ext_over_phi0_scan, y=Ic_patterns[name], mode='lines',
                             name=f'{name} - Ic($\Phi$)', legendgroup=name, 
                             showlegend=False, 
                             line=dict(color=color)),
                  row=2, col=1)

# 添加數學表達式文本
math_text = ""
for i, (name, data) in enumerate(current_distributions.items()):
    math_text += f"<b>{i+1}. {name}:</b><br>{data['math']}<br><br>"

fig.add_annotation(
    text=math_text,
    xref="paper", yref="paper",
    x=0.05, y=0.28,
    xanchor="left", yanchor="top",
    showarrow=False,
    font=dict(size=11),
    bgcolor="rgba(255,255,255,0.8)",
    bordercolor="rgba(0,0,0,0.2)",
    borderwidth=1
)

# 更新軸標籤和範圍
fig.update_xaxes(title_text="歸一化接面位置 $x/W$", row=1, col=1)
fig.update_yaxes(title_text="歸一化有效電流密度 $J_{eff}(x)$", row=1, col=1, range=[-1.1, 1.1]) 

fig.update_xaxes(title_text="歸一化外部磁通量 ($\Phi_{ext} / \Phi_0$)", row=2, col=1, dtick=1)
fig.update_yaxes(title_text="歸一化臨界電流 ($I_c / I_{c,peak}$)", row=2, col=1, range=[-0.05, 1.1]) 

# 隱藏第三個子圖的軸
fig.update_xaxes(visible=False, row=3, col=1)
fig.update_yaxes(visible=False, row=3, col=1)

fig.update_layout(
    height=1800, 
    title_text="指定電流密度分佈及其對應的夫琅禾費干涉圖樣與數學表達式",
    title_x=0.5,
    legend_title_text="有效電流密度分佈類型",
    legend_tracegroupgap = 5 
)

fig.show()

In [2]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.fft import fft, fftshift
from scipy.special import jv, yv # Bessel functions J_n and Y_n
from scipy.interpolate import lagrange # Lagrange polynomials

# --- 常數與參數設定 ---
Phi0 = 1.0       # 磁通量量子 (歸一化)
W = 1.0          # 接面寬度 (歸一化)
N_points_x = 256 # 接面空間取樣點數
x_coords = np.linspace(-W/2, W/2, N_points_x) # 接面寬度座標

# 掃描的外部磁通量範圍 (歸一化)
phi_ext_over_phi0_scan = np.linspace(-8, 8, 800) # 擴大範圍以觀察更多旁瓣

# --- 定義不同的電流密度分佈函數 Jc(x) ---
def uniform_Jc(x_coords, W_val):
    """均勻電流密度"""
    return np.ones_like(x_coords)

def symmetric_edge_gaussian_Jc(x_coords, W_val, peak_width_factor=0.1, edge_offset_factor=0.05):
    """對稱邊緣集中的電流密度 (兩個高斯峰)"""
    sigma = W_val * peak_width_factor
    peak1_center = -W_val/2 + W_val * edge_offset_factor + sigma 
    peak2_center = W_val/2 - W_val * edge_offset_factor - sigma
    return np.exp(-(x_coords - peak1_center)**2 / (2 * sigma**2)) + \
           np.exp(-(x_coords - peak2_center)**2 / (2 * sigma**2))

def asymmetric_linear_Jc(x_coords, W_val):
    """不對稱線性梯度電流密度 (僅正值)"""
    return 0.7 + 0.5 * (x_coords / (W_val/2)) 

def one_side_step_Jc(x_coords, W_val, step_start_norm=-0.5, step_end_norm=-0.2):
    """只有一邊的不對稱邊緣集中 (階梯)"""
    J = np.zeros_like(x_coords)
    start_x = step_start_norm * W_val
    end_x = step_end_norm * W_val
    actual_start = np.maximum(start_x, -W_val/2)
    actual_end = np.minimum(end_x, W_val/2)
    if actual_start < actual_end:
         J[(x_coords >= actual_start) & (x_coords <= actual_end)] = 1.0
    return J

def asymmetric_multistep_Jc(x_coords, W_val, x_split_norm=-0.1, h1=1.0, h2=0.4):
    """不對稱多階梯 (兩段不同高度, 僅正值)"""
    J = np.zeros_like(x_coords)
    x_split = x_split_norm * W_val
    J[x_coords <= x_split] = h1
    J[x_coords > x_split] = h2
    J[x_coords < -W_val/2] = 0
    J[x_coords > W_val/2] = 0
    return J

def symmetric_edge_steps_Jc(x_coords, W_val, depth_norm=0.2, base_level=0.0):
    """雙邊均勻邊緣集中 (對稱階梯)"""
    J = np.full_like(x_coords, base_level)
    edge_width = depth_norm * W_val
    J[(x_coords >= -W_val/2) & (x_coords <= -W_val/2 + edge_width)] = 1.0
    J[(x_coords >= W_val/2 - edge_width) & (x_coords <= W_val/2)] = 1.0
    return J

def bilateral_opposite_shape_edge_Jc(x_coords, W_val, 
                                     left_step_end_norm=-0.25, left_h=1.0,
                                     right_peak_center_norm=0.3, right_peak_h=0.8, right_peak_sigma_norm=0.07):
    """雙邊異形邊緣集中 (左階梯, 右高斯)"""
    J_left = np.zeros_like(x_coords)
    left_step_end = left_step_end_norm * W_val
    J_left[(x_coords >= -W_val/2) & (x_coords <= left_step_end)] = left_h 

    J_right = np.zeros_like(x_coords)
    right_peak_center = right_peak_center_norm * W_val
    right_sigma = right_peak_sigma_norm * W_val
    J_right = right_peak_h * np.exp(-(x_coords - right_peak_center)**2 / (2 * right_sigma**2)) 

    J_combined = np.zeros_like(x_coords)
    mask_left = (x_coords >= -W_val/2) & (x_coords <= left_step_end)
    mask_right_exclusive = (x_coords > left_step_end) 

    J_combined[mask_left] = J_left[mask_left]
    J_combined[mask_right_exclusive] = J_right[mask_right_exclusive]
    return J_combined

def Jc_with_linear_phase_gradient(x_coords, W_val, J_amplitude_func, phase_k0_norm):
    """帶有內稟線性相位梯度的電流密度 (用於產生位移的對稱圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    k_intrinsic = phase_k0_norm * np.pi 
    return J_real * np.exp(1j * k_intrinsic * (x_coords / (W/2)))

def asymmetric_phi0_profile(x_coords, W_val, A=np.pi/2, x_d_norm=-0.2, sigma_norm=0.15):
    """一個不對稱的內稟相位移 phi_0^{int}(x) 範例 (例如，一側的相位突變)"""
    x_d = x_d_norm * W_val
    sigma = sigma_norm * W_val
    return A * np.exp(-(x_coords - x_d)**2 / (2 * sigma**2))

def Jc_with_asymmetric_phi0(x_coords, W_val, J_amplitude_func, phi0_int_func, phi0_params):
    """帶有不對稱內稟相位移的電流密度 (用於產生不對稱的Ic(Phi)圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    phi0_intrinsic = phi0_int_func(x_coords, W_val, **phi0_params)
    return J_real * np.exp(-1j * phi0_intrinsic) # J_eff = J_c * exp(-i * phi_0_int)

# 選定的電流密度分佈與數學表達式 (使用HTML格式，避免LaTeX渲染問題)
current_distributions = {
    "均勻 (Uniform)": {
        "profile": uniform_Jc(x_coords, W),
        "math": "J<sub>c</sub>(x) = J<sub>0</sub> (constant)"
    },
    "對稱邊緣高斯 (Symm. Edge Gaussian)": {
        "profile": symmetric_edge_gaussian_Jc(x_coords, W, peak_width_factor=0.08, edge_offset_factor=0.05),
        "math": "J<sub>c</sub>(x) = exp(-(x-x<sub>1</sub>)<sup>2</sup>/(2σ<sup>2</sup>)) + exp(-(x-x<sub>2</sub>)<sup>2</sup>/(2σ<sup>2</sup>))"
    },
    "不對稱線性 (僅正值) (Asymm. Linear)": {
        "profile": asymmetric_linear_Jc(x_coords, W),
        "math": "J<sub>c</sub>(x) = 0.7 + 0.5 × (x/(W/2))"
    },
    "單邊階梯 (One-side Step)": {
        "profile": one_side_step_Jc(x_coords, W, step_start_norm=-0.5, step_end_norm=-0.2),
        "math": "J<sub>c</sub>(x) = { J<sub>0</sub> if x<sub>1</sub> ≤ x ≤ x<sub>2</sub>; 0 otherwise }"
    },
    "不對稱多階梯 (僅正值) (Asymm. Multi-Steps)": {
        "profile": asymmetric_multistep_Jc(x_coords, W, x_split_norm=-0.1, h1=1.0, h2=0.4),
        "math": "J<sub>c</sub>(x) = { h<sub>1</sub> if x ≤ x<sub>s</sub>; h<sub>2</sub> if x > x<sub>s</sub> }"
    },
    "雙邊均勻邊緣 (Symm. Edge Steps)": {
        "profile": symmetric_edge_steps_Jc(x_coords, W, depth_norm=0.15, base_level=0.0),
        "math": "J<sub>c</sub>(x) = { J<sub>0</sub> if |x| > W/2 - d; 0 otherwise }"
    },
    "雙邊異形邊緣 (Bilateral Diff. Shapes)": {
        "profile": bilateral_opposite_shape_edge_Jc(x_coords, W, left_step_end_norm=-0.2, left_h=0.9, right_peak_center_norm=0.3, right_peak_h=1.0, right_peak_sigma_norm=0.1),
        "math": "J<sub>c</sub>(x) = { h<sub>L</sub> (left step); h<sub>R</sub> exp(-(x-x<sub>R</sub>)<sup>2</sup>/(2σ<sub>R</sub><sup>2</sup>)) (right Gaussian) }"
    },
    "均勻Jc+線性相位梯度 (僅正值) (Shifted Uniform)": {
        "profile": Jc_with_linear_phase_gradient(x_coords, W, lambda x, w: uniform_Jc(x,w), phase_k0_norm=0.5),
        "math": "J<sub>eff</sub>(x) = J<sub>0</sub> exp(i k<sub>0</sub> x)"
    },
    "均勻Jc+不對稱相位 (僅正值) (Asymmetric Ic(Phi))": {
        "profile": Jc_with_asymmetric_phi0(x_coords, W, 
                                              lambda x, w: uniform_Jc(x,w), # 基底 Jc(x)
                                              asymmetric_phi0_profile,     # 不對稱 phi_0^{int}(x) 函數
                                              {'A': np.pi*0.8, 'x_d_norm': -0.25, 'sigma_norm': 0.2}), # phi_0^{int}(x) 的參數
        "math": "J<sub>eff</sub>(x) = J<sub>0</sub> exp(-i φ<sub>0</sub><sup>int</sup>(x)),<br>where φ<sub>0</sub><sup>int</sup>(x) = A exp(-(x-x<sub>d</sub>)<sup>2</sup>/(2σ<sup>2</sup>))"
    }
}

# 歸一化電流密度分佈
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    if np.iscomplexobj(Jc_profile_complex):
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9:
            current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val
    else: 
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9: 
             current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val 

# 計算 Ic(Phi) 圖樣
Ic_patterns = {}
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    Ic_vs_phi = []
    for phi_norm in phi_ext_over_phi0_scan:
        integrand = Jc_profile_complex * np.exp(1j * 2 * np.pi * phi_norm * (x_coords / W))
        dx = W / (N_points_x -1) if N_points_x > 1 else W 
        Ic_val = np.abs(np.sum(integrand) * dx) 
        Ic_vs_phi.append(Ic_val)

    Ic_patterns[name] = np.array(Ic_vs_phi)
    max_ic_val_pattern = np.max(Ic_patterns[name])
    if max_ic_val_pattern > 1e-9:
         Ic_patterns[name] /= max_ic_val_pattern

# 創建圖表 - 方法1: 使用HTML格式的數學表達式
fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=False,
                    row_heights=[0.35, 0.35, 0.3],
                    subplot_titles=("電流密度分佈 J<sub>eff</sub>(x) (實部與虛部，已歸一化至|·|<sub>max</sub>=1)", 
                                    "對應的夫琅禾費圖樣 I<sub>c</sub>(Φ<sub>ext</sub>) (已歸一化至I<sub>c,peak</sub>=1)",
                                    "數學表達式"))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# 繪製電流密度分佈
for i, (name, data) in enumerate(current_distributions.items()):
    Jc_profile_complex = data["profile"]
    color = colors[i % len(colors)]
    
    if np.iscomplexobj(Jc_profile_complex):
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.real(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (實部)', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.imag(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (虛部)', legendgroup=name, 
                                 line=dict(color=color, dash='dot')),
                      row=1, col=1)
    else:
        fig.add_trace(go.Scatter(x=x_coords/W, y=Jc_profile_complex, mode='lines', 
                                 name=f'{name}', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)

    # 繪製 Ic(Phi) 圖樣
    fig.add_trace(go.Scatter(x=phi_ext_over_phi0_scan, y=Ic_patterns[name], mode='lines',
                             name=f'{name} - Ic(Φ)', legendgroup=name, 
                             showlegend=False, 
                             line=dict(color=color)),
                  row=2, col=1)

# 方法1: 使用HTML格式添加數學表達式文本
math_text_html = ""
for i, (name, data) in enumerate(current_distributions.items()):
    math_text_html += f"<b>{i+1}. {name}:</b><br>{data['math']}<br><br>"

fig.add_annotation(
    text=math_text_html,
    xref="paper", yref="paper",
    x=0.05, y=0.28,
    xanchor="left", yanchor="top",
    showarrow=False,
    font=dict(size=11),
    bgcolor="rgba(255,255,255,0.9)",
    bordercolor="rgba(0,0,0,0.3)",
    borderwidth=1
)

# 更新軸標籤和範圍
fig.update_xaxes(title_text="歸一化接面位置 x/W", row=1, col=1)
fig.update_yaxes(title_text="歸一化有效電流密度 J<sub>eff</sub>(x)", row=1, col=1, range=[-1.1, 1.1]) 

fig.update_xaxes(title_text="歸一化外部磁通量 (Φ<sub>ext</sub> / Φ<sub>0</sub>)", row=2, col=1, dtick=1)
fig.update_yaxes(title_text="歸一化臨界電流 (I<sub>c</sub> / I<sub>c,peak</sub>)", row=2, col=1, range=[-0.05, 1.1]) 

# 隱藏第三個子圖的軸
fig.update_xaxes(visible=False, row=3, col=1)
fig.update_yaxes(visible=False, row=3, col=1)

# 設定 MathJax 支援 (如果需要LaTeX渲染)
fig.update_layout(
    height=1800, 
    title_text="指定電流密度分佈及其對應的夫琅禾費干涉圖樣與數學表達式",
    title_x=0.5,
    legend_title_text="有效電流密度分佈類型",
    legend_tracegroupgap = 5,
    # 嘗試啟用MathJax支援
    font=dict(family="Arial", size=12)
)

# 如果您希望使用LaTeX格式，可以嘗試以下設定：
# fig.update_layout(
#     font=dict(family="Computer Modern"),  # LaTeX字體
#     title_font_size=16,
# )

fig.show()

# 或者，如果上述方法仍有問題，可以使用以下替代方案：
print("如果圖表中的數學表達式仍有顯示問題，請嘗試以下解決方案：")
print("1. 更新 Plotly: pip install --upgrade plotly")
print("2. 在 Jupyter 中啟用 MathJax: %config InlineBackend.figure_format = 'retina'")
print("3. 或使用純文字輸出數學表達式：")
print("\n數學表達式列表：")
for i, (name, data) in enumerate(current_distributions.items()):
    print(f"{i+1}. {name}: {data['math']}")

如果圖表中的數學表達式仍有顯示問題，請嘗試以下解決方案：
1. 更新 Plotly: pip install --upgrade plotly
2. 在 Jupyter 中啟用 MathJax: %config InlineBackend.figure_format = 'retina'
3. 或使用純文字輸出數學表達式：

數學表達式列表：
1. 均勻 (Uniform): J<sub>c</sub>(x) = J<sub>0</sub> (constant)
2. 對稱邊緣高斯 (Symm. Edge Gaussian): J<sub>c</sub>(x) = exp(-(x-x<sub>1</sub>)<sup>2</sup>/(2σ<sup>2</sup>)) + exp(-(x-x<sub>2</sub>)<sup>2</sup>/(2σ<sup>2</sup>))
3. 不對稱線性 (僅正值) (Asymm. Linear): J<sub>c</sub>(x) = 0.7 + 0.5 × (x/(W/2))
4. 單邊階梯 (One-side Step): J<sub>c</sub>(x) = { J<sub>0</sub> if x<sub>1</sub> ≤ x ≤ x<sub>2</sub>; 0 otherwise }
5. 不對稱多階梯 (僅正值) (Asymm. Multi-Steps): J<sub>c</sub>(x) = { h<sub>1</sub> if x ≤ x<sub>s</sub>; h<sub>2</sub> if x > x<sub>s</sub> }
6. 雙邊均勻邊緣 (Symm. Edge Steps): J<sub>c</sub>(x) = { J<sub>0</sub> if |x| > W/2 - d; 0 otherwise }
7. 雙邊異形邊緣 (Bilateral Diff. Shapes): J<sub>c</sub>(x) = { h<sub>L</sub> (left step); h<sub>R</sub> exp(-(x-x<sub>R</sub>)<sup>2</sup>/(2σ<sub>R</sub><sup>2</sup>)) (right Gaussian) }
8. 均勻Jc+線性

In [3]:
# 方法2: 使用表格形式顯示數學表達式
import pandas as pd

# 創建數學表達式表格
math_expressions_df = pd.DataFrame([
    {"分佈類型": name, "數學表達式": data['math']} 
    for name, data in current_distributions.items()
])

print("數學表達式表格：")
print(math_expressions_df.to_string(index=False))

# 或者使用 Plotly 表格
from plotly.graph_objects import Table

table_fig = go.Figure(data=[Table(
    header=dict(values=['分佈類型', '數學表達式'],
                fill_color='paleturquoise',
                align='left'),
    cells=dict(values=[list(math_expressions_df['分佈類型']), 
                      list(math_expressions_df['數學表達式'])],
               fill_color='lavender',
               align='left'))
])

table_fig.update_layout(title="電流密度分佈的數學表達式")
table_fig.show()

數學表達式表格：
                                 分佈類型                                                                                                                                                                       數學表達式
                         均勻 (Uniform)                                                                                                                                 J<sub>c</sub>(x) = J<sub>0</sub> (constant)
         對稱邊緣高斯 (Symm. Edge Gaussian)                                              J<sub>c</sub>(x) = exp(-(x-x<sub>1</sub>)<sup>2</sup>/(2σ<sup>2</sup>)) + exp(-(x-x<sub>2</sub>)<sup>2</sup>/(2σ<sup>2</sup>))
          不對稱線性 (僅正值) (Asymm. Linear)                                                                                                                                    J<sub>c</sub>(x) = 0.7 + 0.5 × (x/(W/2))
                 單邊階梯 (One-side Step)                                                                                      J<sub>c</sub>(x) = { J<sub>0

In [5]:
import plotly.graph_objects as go
import numpy as np

# --- Parameters ---
J0_amplitude = 1.0  # Amplitude of J0 (assuming J0 is real for simplicity)
k0 = 1.0            # Wavenumber
x_min = 0
x_max = 4 * np.pi   # Plot over a few wavelengths
num_points = 200    # Number of points to plot

# --- Generate x values ---
x_values = np.linspace(x_min, x_max, num_points)

# --- Calculate J_eff(x) ---
# J_eff(x) = J0 * exp(i * k0 * x)
# Using Euler's formula: exp(i * theta) = cos(theta) + i * sin(theta)
# So, J_eff(x) = J0 * (cos(k0 * x) + i * sin(k0 * x))

j_eff_values = J0_amplitude * np.exp(1j * k0 * x_values)

# Extract real, imaginary parts, and magnitude
real_part = j_eff_values.real
imaginary_part = j_eff_values.imag
magnitude = np.abs(j_eff_values) # Should be constant |J0_amplitude|
phase_rad = np.angle(j_eff_values) # Phase in radians
phase_deg = np.rad2deg(phase_rad)  # Phase in degrees

# --- Create the Plotly figure ---
fig = go.Figure()

# Add trace for the Real Part
fig.add_trace(go.Scatter(
    x=x_values,
    y=real_part,
    mode='lines',
    name='Real Part: J₀ cos(k₀x)',
    line=dict(color='blue')
))

# Add trace for the Imaginary Part
fig.add_trace(go.Scatter(
    x=x_values,
    y=imaginary_part,
    mode='lines',
    name='Imaginary Part: J₀ sin(k₀x)',
    line=dict(color='red')
))

# Add trace for the Magnitude
fig.add_trace(go.Scatter(
    x=x_values,
    y=magnitude,
    mode='lines',
    name='Magnitude: |J₀|',
    line=dict(color='green', dash='dash')
))

# Add trace for the Phase (optional, can make plot busy)
# fig.add_trace(go.Scatter(
#     x=x_values,
#     y=phase_deg, # or phase_rad
#     mode='lines',
#     name='Phase (degrees)',
#     line=dict(color='purple', dash='dot'),
#     yaxis="y2" # Assign to a secondary y-axis if desired
# ))

# --- Update layout ---
title_text = f"Complex Plane Wave: J<sub>eff</sub>(x) = J₀ e<sup>ik₀x</sup><br>J₀ = {J0_amplitude}, k₀ = {k0}"
fig.update_layout(
    title=title_text,
    xaxis_title="Position (x)",
    yaxis_title="Amplitude / Magnitude",
    legend_title="Components",
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="RebeccaPurple"
    ),
    # If using phase on a secondary axis:
    # yaxis2=dict(
    #     title="Phase (degrees)",
    #     overlaying="y",
    #     side="right"
    # )
)

# --- Show the plot ---
# In a script, use fig.show().
# If running in an environment like Jupyter, this will display the plot directly.
# To save as HTML: fig.write_html("complex_wave_plot.html")
fig.show()

# --- Alternative: Plot in the Complex Plane (parametric plot) ---
# This shows the circular path of J_eff(x) in the complex plane as x varies.
fig_complex_plane = go.Figure()

fig_complex_plane.add_trace(go.Scatter(
    x=real_part,
    y=imaginary_part,
    mode='lines',
    name=f'J_eff(x) for J₀={J0_amplitude}, k₀={k0}',
    line=dict(color='orange'),
    text=[f"x = {val:.2f}" for val in x_values], # Show x value on hover
    hoverinfo='text+x+y'
))

# Add a marker for the start point
fig_complex_plane.add_trace(go.Scatter(
    x=[real_part[0]],
    y=[imaginary_part[0]],
    mode='markers',
    name='Start (x=0)',
    marker=dict(color='black', size=10, symbol='circle-open')
))


fig_complex_plane.update_layout(
    title=f"J<sub>eff</sub>(x) in the Complex Plane<br>J₀ = {J0_amplitude}, k₀ = {k0}",
    xaxis_title="Real Part",
    yaxis_title="Imaginary Part",
    legend_title="Path",
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="RebeccaPurple"
    ),
    width=700,
    height=700,
    yaxis_scaleanchor="x", # Ensures a circular plot if aspect ratio is 1
    xaxis_constrain="domain",
    yaxis_constrain="domain"

)
# fig_complex_plane.show() # Uncomment to show this plot

# --- Alternative: 3D Plot (helix) ---
# x_coords, Real Part, Imaginary Part
fig_3d = go.Figure()

fig_3d.add_trace(go.Scatter3d(
    x=x_values,
    y=real_part,
    z=imaginary_part,
    mode='lines',
    name=f'J_eff(x) helix for J₀={J0_amplitude}, k₀={k0}',
    line=dict(color='cyan', width=4)
))

fig_3d.update_layout(
    title=f"3D Representation of J<sub>eff</sub>(x) (Helix)<br>J₀ = {J0_amplitude}, k₀ = {k0}",
    scene=dict(
        xaxis_title="Position (x)",
        yaxis_title="Real Part",
        zaxis_title="Imaginary Part"
    ),
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="RebeccaPurple"
    ),
    margin=dict(l=0, r=0, b=0, t=40) # Adjust margins
)
fig_3d.show() # Uncomment to show this plot


In [18]:
import plotly.graph_objects as go
import numpy as np

# --- Parameters ---
J0_amplitude = 0.5  # Amplitude of J0 (assuming J0 is real for simplicity)
k0 = 0.0            # Wavenumber for the J0 term
C_real = 0.5        # Real part of the constant C
C_imag = 0.5        # Imaginary part of the constant C
x_min = 0
x_max = 4 * np.pi   # Spatial extent of the current distribution
num_points_x = 201    # Number of points to plot J_eff(x)

# --- Generate x values for J_eff(x) plotting ---
x_values = np.linspace(x_min, x_max, num_points_x)

# --- Calculate J_eff(x) ---
# J_eff(x) = J0 * exp(i * k0 * x) + C
constant_C = C_real + 1j * C_imag
j_eff_values = J0_amplitude * np.exp(1j * k0 * x_values) + constant_C

# Extract real, imaginary parts, and magnitude for J_eff(x) plotting
real_part_jeff = j_eff_values.real
imaginary_part_jeff = j_eff_values.imag
magnitude_jeff = np.abs(j_eff_values) # Magnitude will now change with x
# phase_rad_jeff = np.angle(j_eff_values) # Phase in radians
# phase_deg_jeff = np.rad2deg(phase_rad_jeff)  # Phase in degrees

# --- Create the Plotly figure for J_eff(x) ---
fig_jeff = go.Figure()

# Add trace for the Real Part of J_eff(x)
fig_jeff.add_trace(go.Scatter(
    x=x_values,
    y=real_part_jeff,
    mode='lines',
    name='Re(J<sub>eff</sub>): J₀ cos(k₀x) + Re(C)',
    line=dict(color='blue')
))

# Add trace for the Imaginary Part of J_eff(x)
fig_jeff.add_trace(go.Scatter(
    x=x_values,
    y=imaginary_part_jeff,
    mode='lines',
    name='Im(J<sub>eff</sub>): J₀ sin(k₀x) + Im(C)',
    line=dict(color='red')
))

# Add trace for the Magnitude of J_eff(x)
fig_jeff.add_trace(go.Scatter(
    x=x_values,
    y=magnitude_jeff,
    mode='lines',
    name='|J<sub>eff</sub>(x)|: |J₀e<sup>ik₀x</sup> + C|',
    line=dict(color='green', dash='dash')
))

# --- Update layout for J_eff(x) plot ---
title_text_jeff = (f"Current Density Distribution: J<sub>eff</sub>(x) = J₀e<sup>ik₀x</sup> + C<br>"
                   f"J₀ = {J0_amplitude}, k₀ = {k0}, C = {C_real:.2f} + {C_imag:.2f}i")
fig_jeff.update_layout(
    title=title_text_jeff,
    xaxis_title="Position (x)",
    yaxis_title="Amplitude / Magnitude",
    legend_title="Components of J<sub>eff</sub>(x)",
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="RebeccaPurple"
    )
)

# --- Show the J_eff(x) plot ---
# fig_jeff.show() # Uncomment to show this plot

# --- Alternative: Plot J_eff(x) in the Complex Plane (parametric plot) ---
fig_complex_plane_jeff = go.Figure()
fig_complex_plane_jeff.add_trace(go.Scatter(
    x=real_part_jeff,
    y=imaginary_part_jeff,
    mode='lines',
    name=f'J_eff(x) for J₀={J0_amplitude}, k₀={k0}, C={C_real:.2f}+{C_imag:.2f}i',
    line=dict(color='orange'),
    text=[f"x = {val:.2f}" for val in x_values],
    hoverinfo='text+x+y'
))
fig_complex_plane_jeff.add_trace(go.Scatter(
    x=[real_part_jeff[0]],
    y=[imaginary_part_jeff[0]],
    mode='markers',
    name=f'Start (x={x_min:.1f})',
    marker=dict(color='black', size=10, symbol='circle-open')
))
fig_complex_plane_jeff.add_trace(go.Scatter(
    x=[C_real],
    y=[C_imag],
    mode='markers',
    name=f'Center C ({C_real:.2f}, {C_imag:.2f})',
    marker=dict(color='purple', size=8, symbol='cross')
))
fig_complex_plane_jeff.update_layout(
    title=(f"J<sub>eff</sub>(x) = J₀e<sup>ik₀x</sup> + C in the Complex Plane<br>"
           f"J₀ = {J0_amplitude}, k₀ = {k0}, C = {C_real:.2f} + {C_imag:.2f}i"),
    xaxis_title="Real Part of J<sub>eff</sub>(x)",
    yaxis_title="Imaginary Part of J<sub>eff</sub>(x)",
    legend_title="Path",
    font=dict(family="Arial, sans-serif", size=12, color="RebeccaPurple"),
    width=700, height=700, yaxis_scaleanchor="x", xaxis_constrain="domain", yaxis_constrain="domain"
)
# fig_complex_plane_jeff.show() # Uncomment to show this plot

# --- Alternative: 3D Plot of J_eff(x) (helix) ---
fig_3d_jeff = go.Figure()
fig_3d_jeff.add_trace(go.Scatter3d(
    x=x_values,
    y=real_part_jeff,
    z=imaginary_part_jeff,
    mode='lines',
    name=f'J_eff(x) helix for J₀={J0_amplitude}, k₀={k0}, C={C_real:.2f}+{C_imag:.2f}i',
    line=dict(color='cyan', width=4)
))
fig_3d_jeff.update_layout(
    title=(f"3D Representation: J<sub>eff</sub>(x) = J₀e<sup>ik₀x</sup> + C (Shifted Helix)<br>"
           f"J₀ = {J0_amplitude}, k₀ = {k0}, C = {C_real:.2f} + {C_imag:.2f}i"),
    scene=dict(xaxis_title="Position (x)", yaxis_title="Real Part of J<sub>eff</sub>(x)", zaxis_title="Imaginary Part of J<sub>eff</sub>(x)"),
    font=dict(family="Arial, sans-serif", size=12, color="RebeccaPurple"),
    margin=dict(l=0, r=0, b=0, t=50)
)
# fig_3d_jeff.show() # Uncomment to show this plot


# --- Calculate Fraunhofer Diffraction Pattern ---
# The Fraunhofer pattern is the squared magnitude of the Fourier Transform of J_eff(x).
# F(k_s) = integral from x_min to x_max of J_eff(x) * exp(-i * k_s * x) dx
# k_s is the spatial frequency (related to diffraction angle)

num_points_ks = 401 # Number of points for the k_s axis

# Define a range for k_s. This should cover the main features.
# The characteristic width of sinc functions is related to 2*pi / (x_max - x_min)
L = x_max - x_min
ks_range_extension = 5 * (2 * np.pi / L if L > 0 else 1.0) # Extend a few sidelobes
ks_center_1 = k0
ks_center_0 = 0.0
ks_min_plot = min(ks_center_0, ks_center_1) - ks_range_extension
ks_max_plot = max(ks_center_0, ks_center_1) + ks_range_extension
if k0 == 0: # Special handling if k0 is 0
    ks_min_plot = -ks_range_extension * 2
    ks_max_plot = ks_range_extension * 2
if L == 0: # If it's a point source (or effectively)
    ks_min_plot = -5.0
    ks_max_plot = 5.0


ks_values = np.linspace(ks_min_plot, ks_max_plot, num_points_ks)

# Helper function for the integral of exp(i*beta*x) from a to b
def integrate_exp_complex(beta, a, b):
    # Numerically stable for beta close to 0
    if np.isclose(beta, 0.0):
        return b - a
    else:
        return (np.exp(1j * beta * b) - np.exp(1j * beta * a)) / (1j * beta)

# Calculate Fourier Transform F(k_s)
F_ks = np.zeros(num_points_ks, dtype=complex)

for i, ks in enumerate(ks_values):
    # Term from J0 * exp(i * k0 * x)
    term1_beta = k0 - ks
    integral1 = J0_amplitude * integrate_exp_complex(term1_beta, x_min, x_max)
    
    # Term from C
    term2_beta = -ks
    integral2 = constant_C * integrate_exp_complex(term2_beta, x_min, x_max)
    
    F_ks[i] = integral1 + integral2

# Intensity of the Fraunhofer pattern
Intensity_ks = np.abs(F_ks)**2

# --- Create Plotly figure for Fraunhofer Pattern ---
fig_fraunhofer = go.Figure()

fig_fraunhofer.add_trace(go.Scatter(
    x=ks_values,
    y=Intensity_ks,
    mode='lines',
    name='Intensity |F(k<sub>s</sub>)|²',
    line=dict(color='purple')
))

fig_fraunhofer.update_layout(
    title=(f"Fraunhofer Diffraction Pattern of J<sub>eff</sub>(x)<br>"
           f"J₀ = {J0_amplitude}, k₀ = {k0}, C = {C_real:.2f} + {C_imag:.2f}i, x ∈ [{x_min:.2f}, {x_max:.2f}]"),
    xaxis_title="Spatial Frequency (k<sub>s</sub>)",
    yaxis_title="Intensity |F(k<sub>s</sub>)|²",
    legend_title="Pattern",
    font=dict(
        family="Arial, sans-serif",
        size=12,
        color="DarkSlateGray"
    )
)

# --- Show the Fraunhofer plot ---
fig_fraunhofer.show() # This will be the primary plot shown



In [25]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.fft import fft, fftshift
from scipy.special import jv, yv # Bessel functions J_n and Y_n
from scipy.interpolate import lagrange # Lagrange polynomials

# --- 常數與參數設定 ---
Phi0 = 1.0       # 磁通量量子 (歸一化)
W = 1.0          # 接面寬度 (歸一化)
N_points_x = 256 # 接面空間取樣點數
x_coords = np.linspace(-W/2, W/2, N_points_x) # 接面寬度座標

# 掃描的外部磁通量範圍 (歸一化)
phi_ext_over_phi0_scan = np.linspace(-8, 8, 800) # 擴大範圍以觀察更多旁瓣

# --- Parameters ---
J0_amplitude = 1.0  # Amplitude of J0 (assuming J0 is real for simplicity)
k0 = 1.0            # Wavenumber
x_min = 0
x_max = 4 * np.pi   # Plot over a few wavelengths
num_points = 201    # Number of points to plot

# --- Generate x values ---
x_values = np.linspace(x_min, x_max, num_points)

# --- Calculate J_eff(x) ---
# J_eff(x) = J0 * exp(i * k0 * x)
# Using Euler's formula: exp(i * theta) = cos(theta) + i * sin(theta)
# So, J_eff(x) = J0 * (cos(k0 * x) + i * sin(k0 * x))

# --- Parameters ---
J0_amplitude = 0.5  # Amplitude of J0 (assuming J0 is real for simplicity)
k0 = 1.0            # Wavenumber for the J0 term
C_real = 0.5        # Real part of the constant C
C_imag = 0.5        # Imaginary part of the constant C
x_min = 0
x_max = 4 * np.pi   # Spatial extent of the current distribution
num_points_x = 201    # Number of points to plot J_eff(x)


# --- Calculate J_eff(x) ---
# J_eff(x) = J0 * exp(i * k0 * x) + C
constant_C = C_real + 1j * C_imag
j_eff_values = J0_amplitude * np.exp(1j * k0 * x_values) + constant_C
def J_eff_formula(x, J0, k0, constant_C):
    """
    計算有效電流密度 J_eff(x) = J0 * exp(i * k0 * x) + constant_C
    返回複數值
    """
    return J0 * np.exp(1j * k0 * x) + constant_C

# --- 定義不同的電流密度分佈函數 Jc(x) ---
def uniform_Jc(x_coords, W_val):
    """均勻電流密度"""
    return np.ones_like(x_coords)

def symmetric_edge_gaussian_Jc(x_coords, W_val, peak_width_factor=0.1, edge_offset_factor=0.05):
    """對稱邊緣集中的電流密度 (兩個高斯峰)"""
    sigma = W_val * peak_width_factor
    peak1_center = -W_val/2 + W_val * edge_offset_factor + sigma 
    peak2_center = W_val/2 - W_val * edge_offset_factor - sigma
    return np.exp(-(x_coords - peak1_center)**2 / (2 * sigma**2)) + \
           np.exp(-(x_coords - peak2_center)**2 / (2 * sigma**2))

def asymmetric_linear_Jc(x_coords, W_val):
    """不對稱線性梯度電流密度 (僅正值)"""
    return 0.7 + 0.5 * (x_coords / (W_val/2)) 

def one_side_step_Jc(x_coords, W_val, step_start_norm=-0.5, step_end_norm=-0.2):
    """只有一邊的不對稱邊緣集中 (階梯)"""
    J = np.zeros_like(x_coords)
    start_x = step_start_norm * W_val
    end_x = step_end_norm * W_val
    actual_start = np.maximum(start_x, -W_val/2)
    actual_end = np.minimum(end_x, W_val/2)
    if actual_start < actual_end:
         J[(x_coords >= actual_start) & (x_coords <= actual_end)] = 1.0
    return J

def asymmetric_multistep_Jc(x_coords, W_val, x_split_norm=-0.1, h1=1.0, h2=0.4):
    """不對稱多階梯 (兩段不同高度, 僅正值)"""
    J = np.zeros_like(x_coords)
    x_split = x_split_norm * W_val
    J[x_coords <= x_split] = h1
    J[x_coords > x_split] = h2
    J[x_coords < -W_val/2] = 0
    J[x_coords > W_val/2] = 0
    return J

def symmetric_edge_steps_Jc(x_coords, W_val, depth_norm=0.2, base_level=0.0):
    """雙邊均勻邊緣集中 (對稱階梯)"""
    J = np.full_like(x_coords, base_level)
    edge_width = depth_norm * W_val
    J[(x_coords >= -W_val/2) & (x_coords <= -W_val/2 + edge_width)] = 1.0
    J[(x_coords >= W_val/2 - edge_width) & (x_coords <= W_val/2)] = 1.0
    return J

def bilateral_opposite_shape_edge_Jc(x_coords, W_val, 
                                     left_step_end_norm=-0.25, left_h=1.0,
                                     right_peak_center_norm=0.3, right_peak_h=0.8, right_peak_sigma_norm=0.07):
    """雙邊異形邊緣集中 (左階梯, 右高斯)"""
    J_left = np.zeros_like(x_coords)
    left_step_end = left_step_end_norm * W_val
    J_left[(x_coords >= -W_val/2) & (x_coords <= left_step_end)] = left_h 

    J_right = np.zeros_like(x_coords)
    right_peak_center = right_peak_center_norm * W_val
    right_sigma = right_peak_sigma_norm * W_val
    J_right = right_peak_h * np.exp(-(x_coords - right_peak_center)**2 / (2 * right_sigma**2)) 

    J_combined = np.zeros_like(x_coords)
    mask_left = (x_coords >= -W_val/2) & (x_coords <= left_step_end)
    mask_right_exclusive = (x_coords > left_step_end) 

    J_combined[mask_left] = J_left[mask_left]
    J_combined[mask_right_exclusive] = J_right[mask_right_exclusive]
    return J_combined

def Jc_with_linear_phase_gradient(x_coords, W_val, J_amplitude_func, phase_k0_norm):
    """帶有內稟線性相位梯度的電流密度 (用於產生位移的對稱圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    k_intrinsic = phase_k0_norm * np.pi 
    return J_real * np.exp(1j * k_intrinsic * (x_coords / (W/2)))

def Jc_with_linear_phase_gradient_new(x_coords, W_val, J0=1.0, k0_val=1.0, constant_C = C_real + 1j * C_imag):
    """使用新參數定義的線性相位梯度電流密度"""
    # 將 x_coords 從 [-W/2, W/2] 映射到 [x_min, x_max] 範圍
    x_mapped = (x_coords + W_val/2) / W_val * (x_max - x_min) + x_min
    return J_eff_formula(x_mapped, J0, k0_val, constant_C )

def asymmetric_phi0_profile(x_coords, W_val, A=np.pi/2, x_d_norm=-0.2, sigma_norm=0.15):
    """一個不對稱的內稟相位移 phi_0^{int}(x) 範例 (例如，一側的相位突變)"""
    x_d = x_d_norm * W_val
    sigma = sigma_norm * W_val
    return A * np.exp(-(x_coords - x_d)**2 / (2 * sigma**2))

def Jc_with_asymmetric_phi0(x_coords, W_val, J_amplitude_func, phi0_int_func, phi0_params):
    """帶有不對稱內稟相位移的電流密度 (用於產生不對稱的Ic(Phi)圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    phi0_intrinsic = phi0_int_func(x_coords, W_val, **phi0_params)
    return J_real * np.exp(-1j * phi0_intrinsic) # J_eff = J_c * exp(-i * phi_0_int)

# 選定的電流密度分佈與數學表達式 (使用HTML格式，避免LaTeX渲染問題)
current_distributions = {
    "均勻 (Uniform)": {
        "profile": uniform_Jc(x_coords, W),
        "math": "J<sub>c</sub>(x) = J<sub>0</sub> (constant)"
    },
    "對稱邊緣高斯 (Symm. Edge Gaussian)": {
        "profile": symmetric_edge_gaussian_Jc(x_coords, W, peak_width_factor=0.08, edge_offset_factor=0.05),
        "math": "J<sub>c</sub>(x) = exp(-(x-x<sub>1</sub>)<sup>2</sup>/(2σ<sup>2</sup>)) + exp(-(x-x<sub>2</sub>)<sup>2</sup>/(2σ<sup>2</sup>))"
    },
    "不對稱線性 (僅正值) (Asymm. Linear)": {
        "profile": asymmetric_linear_Jc(x_coords, W),
        "math": "J<sub>c</sub>(x) = 0.7 + 0.5 × (x/(W/2))"
    },
    "單邊階梯 (One-side Step)": {
        "profile": one_side_step_Jc(x_coords, W, step_start_norm=-0.5, step_end_norm=-0.2),
        "math": "J<sub>c</sub>(x) = { J<sub>0</sub> if x<sub>1</sub> ≤ x ≤ x<sub>2</sub>; 0 otherwise }"
    },
    "不對稱多階梯 (僅正值) (Asymm. Multi-Steps)": {
        "profile": asymmetric_multistep_Jc(x_coords, W, x_split_norm=-0.1, h1=1.0, h2=0.4),
        "math": "J<sub>c</sub>(x) = { h<sub>1</sub> if x ≤ x<sub>s</sub>; h<sub>2</sub> if x > x<sub>s</sub> }"
    },
    "雙邊均勻邊緣 (Symm. Edge Steps)": {
        "profile": symmetric_edge_steps_Jc(x_coords, W, depth_norm=0.15, base_level=0.0),
        "math": "J<sub>c</sub>(x) = { J<sub>0</sub> if |x| > W/2 - d; 0 otherwise }"
    },
    "雙邊異形邊緣 (Bilateral Diff. Shapes)": {
        "profile": bilateral_opposite_shape_edge_Jc(x_coords, W, left_step_end_norm=-0.2, left_h=0.9, right_peak_center_norm=0.3, right_peak_h=1.0, right_peak_sigma_norm=0.1),
        "math": "J<sub>c</sub>(x) = { h<sub>L</sub> (left step); h<sub>R</sub> exp(-(x-x<sub>R</sub>)<sup>2</sup>/(2σ<sub>R</sub><sup>2</sup>)) (right Gaussian) }"
    },
    "均勻Jc+線性相位梯度 (僅正值) (Shifted Uniform)": {
        "profile": Jc_with_linear_phase_gradient(x_coords, W, lambda x, w: uniform_Jc(x,w), phase_k0_norm=0.5),
        "math": "J<sub>eff</sub>(x) = J<sub>0</sub> exp(i k<sub>0</sub> x)"
    },
    "新定義線性相位梯度 (New Linear Phase)": {
        "profile": Jc_with_linear_phase_gradient_new(x_coords, W, J0_amplitude, k0, constant_C),
        "math": f"J<sub>eff</sub>(x) = {J0_amplitude} × exp(i × {k0} × x)<br>= {J0_amplitude} × (cos({k0}x) + i × sin({k0}x))"
    },
    "均勻Jc+不對稱相位 (僅正值) (Asymmetric Ic(Phi))": {
        "profile": Jc_with_asymmetric_phi0(x_coords, W, 
                                              lambda x, w: uniform_Jc(x,w), # 基底 Jc(x)
                                              asymmetric_phi0_profile,     # 不對稱 phi_0^{int}(x) 函數
                                              {'A': np.pi*0.8, 'x_d_norm': -0.25, 'sigma_norm': 0.2}), # phi_0^{int}(x) 的參數
        "math": "J<sub>eff</sub>(x) = J<sub>0</sub> exp(-i φ<sub>0</sub><sup>int</sup>(x)),<br>where φ<sub>0</sub><sup>int</sup>(x) = A exp(-(x-x<sub>d</sub>)<sup>2</sup>/(2σ<sup>2</sup>))"
    }
}

# 歸一化電流密度分佈
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    if np.iscomplexobj(Jc_profile_complex):
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9:
            current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val
    else: 
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9: 
             current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val 

# 計算 Ic(Phi) 圖樣
Ic_patterns = {}
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    Ic_vs_phi = []
    for phi_norm in phi_ext_over_phi0_scan:
        integrand = Jc_profile_complex * np.exp(1j * 2 * np.pi * phi_norm * (x_coords / W))
        dx = W / (N_points_x -1) if N_points_x > 1 else W 
        Ic_val = np.abs(np.sum(integrand) * dx) 
        Ic_vs_phi.append(Ic_val)

    Ic_patterns[name] = np.array(Ic_vs_phi)
    max_ic_val_pattern = np.max(Ic_patterns[name])
    if max_ic_val_pattern > 1e-9:
         Ic_patterns[name] /= max_ic_val_pattern

# 創建主要圖表
fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=False,
                    row_heights=[0.35, 0.35, 0.3],
                    subplot_titles=("電流密度分佈 J<sub>eff</sub>(x) (實部與虛部，已歸一化至|·|<sub>max</sub>=1)", 
                                    "對應的夫琅禾費圖樣 I<sub>c</sub>(Φ<sub>ext</sub>) (已歸一化至I<sub>c,peak</sub>=1)",
                                    "數學表達式"))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# 繪製電流密度分佈
for i, (name, data) in enumerate(current_distributions.items()):
    Jc_profile_complex = data["profile"]
    color = colors[i % len(colors)]
    
    if np.iscomplexobj(Jc_profile_complex):
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.real(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (實部)', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.imag(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (虛部)', legendgroup=name, 
                                 line=dict(color=color, dash='dot')),
                      row=1, col=1)
    else:
        fig.add_trace(go.Scatter(x=x_coords/W, y=Jc_profile_complex, mode='lines', 
                                 name=f'{name}', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)

    # 繪製 Ic(Phi) 圖樣
    fig.add_trace(go.Scatter(x=phi_ext_over_phi0_scan, y=Ic_patterns[name], mode='lines',
                             name=f'{name} - Ic(Φ)', legendgroup=name, 
                             showlegend=False, 
                             line=dict(color=color)),
                  row=2, col=1)

# 添加數學表達式文本
math_text_html = ""
for i, (name, data) in enumerate(current_distributions.items()):
    math_text_html += f"<b>{i+1}. {name}:</b><br>{data['math']}<br><br>"

fig.add_annotation(
    text=math_text_html,
    xref="paper", yref="paper",
    x=0.05, y=0.28,
    xanchor="left", yanchor="top",
    showarrow=False,
    font=dict(size=10),
    bgcolor="rgba(255,255,255,0.9)",
    bordercolor="rgba(0,0,0,0.3)",
    borderwidth=1
)

# 更新軸標籤和範圍
fig.update_xaxes(title_text="歸一化接面位置 x/W", row=1, col=1)
fig.update_yaxes(title_text="歸一化有效電流密度 J<sub>eff</sub>(x)", row=1, col=1, range=[-1.1, 1.1]) 

fig.update_xaxes(title_text="歸一化外部磁通量 (Φ<sub>ext</sub> / Φ<sub>0</sub>)", row=2, col=1, dtick=1)
fig.update_yaxes(title_text="歸一化臨界電流 (I<sub>c</sub> / I<sub>c,peak</sub>)", row=2, col=1, range=[-0.05, 1.1]) 

# 隱藏第三個子圖的軸
fig.update_xaxes(visible=False, row=3, col=1)
fig.update_yaxes(visible=False, row=3, col=1)

fig.update_layout(
    height=1800, 
    title_text="電流密度分佈及其對應的夫琅禾費干涉圖樣與數學表達式",
    title_x=0.5,
    legend_title_text="有效電流密度分佈類型",
    legend_tracegroupgap = 5,
    font=dict(family="Arial", size=12)
)

fig.show()

# 創建額外的 J_eff(x) 詳細圖表，展示新參數定義的函數
fig_jeff = go.Figure()

# 計算並繪製 J_eff(x) 在完整範圍內的行為
J_eff_values = J_eff_formula(x_values, J0_amplitude, k0)

fig_jeff.add_trace(go.Scatter(x=x_values, y=np.real(J_eff_values), 
                              mode='lines', name='實部 Re[J_eff(x)]',
                              line=dict(color='blue')))
fig_jeff.add_trace(go.Scatter(x=x_values, y=np.imag(J_eff_values), 
                              mode='lines', name='虛部 Im[J_eff(x)]',
                              line=dict(color='red', dash='dash')))
fig_jeff.add_trace(go.Scatter(x=x_values, y=np.abs(J_eff_values), 
                              mode='lines', name='幅度 |J_eff(x)|',
                              line=dict(color='green', width=2)))

fig_jeff.update_layout(
    title=f"J<sub>eff</sub>(x) = {J0_amplitude} × exp(i × {k0} × x) 的詳細行為",
    xaxis_title="x",
    yaxis_title="J<sub>eff</sub>(x)",
    height=600,
    legend=dict(x=0.7, y=0.95)
)

fig_jeff.show()

# 輸出參數信息
print(f"使用的參數:")
print(f"J0_amplitude = {J0_amplitude}")
print(f"k0 = {k0}")
print(f"x範圍: [{x_min}, {x_max}]")
print(f"波長 λ = 2π/k0 = {2*np.pi/k0:.2f}")
print(f"在範圍內包含 {(x_max-x_min)/(2*np.pi/k0):.1f} 個完整波長")

TypeError: J_eff_formula() missing 1 required positional argument: 'constant_C'

In [26]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.fft import fft, fftshift
from scipy.special import jv, yv # Bessel functions J_n and Y_n
from scipy.interpolate import lagrange # Lagrange polynomials

# --- 常數與參數設定 ---
Phi0 = 1.0       # 磁通量量子 (歸一化)
W = 1.0          # 接面寬度 (歸一化)
N_points_x = 256 # 接面空間取樣點數
x_coords = np.linspace(-W/2, W/2, N_points_x) # 接面寬度座標

# 掃描的外部磁通量範圍 (歸一化)
phi_ext_over_phi0_scan = np.linspace(-8, 8, 800) # 擴大範圍以觀察更多旁瓣

# --- Parameters ---
J0_amplitude = 1.0  # Amplitude of J0 (assuming J0 is real for simplicity)
k0 = 1.0            # Wavenumber
x_min = 0
x_max = 4 * np.pi   # Plot over a few wavelengths
num_points = 200    # Number of points to plot

# --- Generate x values ---
x_values = np.linspace(x_min, x_max, num_points)

# --- Calculate J_eff(x) ---
# J_eff(x) = J0 * exp(i * k0 * x)
# Using Euler's formula: exp(i * theta) = cos(theta) + i * sin(theta)
# So, J_eff(x) = J0 * (cos(k0 * x) + i * sin(k0 * x))
def J_eff_formula(x, J0, k0):
    """
    計算有效電流密度 J_eff(x) = J0 * exp(i * k0 * x)
    返回複數值
    """
    return J0 * np.exp(1j * k0 * x)

# --- 定義不同的電流密度分佈函數 Jc(x) ---
def uniform_Jc(x_coords, W_val):
    """均勻電流密度"""
    return np.ones_like(x_coords)

def symmetric_edge_gaussian_Jc(x_coords, W_val, peak_width_factor=0.1, edge_offset_factor=0.05):
    """對稱邊緣集中的電流密度 (兩個高斯峰)"""
    sigma = W_val * peak_width_factor
    peak1_center = -W_val/2 + W_val * edge_offset_factor + sigma 
    peak2_center = W_val/2 - W_val * edge_offset_factor - sigma
    return np.exp(-(x_coords - peak1_center)**2 / (2 * sigma**2)) + \
           np.exp(-(x_coords - peak2_center)**2 / (2 * sigma**2))

def asymmetric_linear_Jc(x_coords, W_val):
    """不對稱線性梯度電流密度 (僅正值)"""
    return 0.7 + 0.5 * (x_coords / (W_val/2)) 

def one_side_step_Jc(x_coords, W_val, step_start_norm=-0.5, step_end_norm=-0.2):
    """只有一邊的不對稱邊緣集中 (階梯)"""
    J = np.zeros_like(x_coords)
    start_x = step_start_norm * W_val
    end_x = step_end_norm * W_val
    actual_start = np.maximum(start_x, -W_val/2)
    actual_end = np.minimum(end_x, W_val/2)
    if actual_start < actual_end:
         J[(x_coords >= actual_start) & (x_coords <= actual_end)] = 1.0
    return J

def asymmetric_multistep_Jc(x_coords, W_val, x_split_norm=-0.1, h1=1.0, h2=0.4):
    """不對稱多階梯 (兩段不同高度, 僅正值)"""
    J = np.zeros_like(x_coords)
    x_split = x_split_norm * W_val
    J[x_coords <= x_split] = h1
    J[x_coords > x_split] = h2
    J[x_coords < -W_val/2] = 0
    J[x_coords > W_val/2] = 0
    return J

def symmetric_edge_steps_Jc(x_coords, W_val, depth_norm=0.2, base_level=0.0):
    """雙邊均勻邊緣集中 (對稱階梯)"""
    J = np.full_like(x_coords, base_level)
    edge_width = depth_norm * W_val
    J[(x_coords >= -W_val/2) & (x_coords <= -W_val/2 + edge_width)] = 1.0
    J[(x_coords >= W_val/2 - edge_width) & (x_coords <= W_val/2)] = 1.0
    return J

def bilateral_opposite_shape_edge_Jc(x_coords, W_val, 
                                     left_step_end_norm=-0.25, left_h=1.0,
                                     right_peak_center_norm=0.3, right_peak_h=0.8, right_peak_sigma_norm=0.07):
    """雙邊異形邊緣集中 (左階梯, 右高斯)"""
    J_left = np.zeros_like(x_coords)
    left_step_end = left_step_end_norm * W_val
    J_left[(x_coords >= -W_val/2) & (x_coords <= left_step_end)] = left_h 

    J_right = np.zeros_like(x_coords)
    right_peak_center = right_peak_center_norm * W_val
    right_sigma = right_peak_sigma_norm * W_val
    J_right = right_peak_h * np.exp(-(x_coords - right_peak_center)**2 / (2 * right_sigma**2)) 

    J_combined = np.zeros_like(x_coords)
    mask_left = (x_coords >= -W_val/2) & (x_coords <= left_step_end)
    mask_right_exclusive = (x_coords > left_step_end) 

    J_combined[mask_left] = J_left[mask_left]
    J_combined[mask_right_exclusive] = J_right[mask_right_exclusive]
    return J_combined

def Jc_with_linear_phase_gradient(x_coords, W_val, J_amplitude_func, phase_k0_norm):
    """帶有內稟線性相位梯度的電流密度 (用於產生位移的對稱圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    k_intrinsic = phase_k0_norm * np.pi 
    return J_real * np.exp(1j * k_intrinsic * (x_coords / (W/2)))

def Jc_with_linear_phase_gradient_new(x_coords, W_val, J0=1.0, k0_val=1.0):
    """使用新參數定義的線性相位梯度電流密度"""
    # 將 x_coords 從 [-W/2, W/2] 映射到 [x_min, x_max] 範圍
    x_mapped = (x_coords + W_val/2) / W_val * (x_max - x_min) + x_min
    return J_eff_formula(x_mapped, J0, k0_val)

def asymmetric_phi0_profile(x_coords, W_val, A=np.pi/2, x_d_norm=-0.2, sigma_norm=0.15):
    """一個不對稱的內稟相位移 phi_0^{int}(x) 範例 (例如，一側的相位突變)"""
    x_d = x_d_norm * W_val
    sigma = sigma_norm * W_val
    return A * np.exp(-(x_coords - x_d)**2 / (2 * sigma**2))

def Jc_with_asymmetric_phi0(x_coords, W_val, J_amplitude_func, phi0_int_func, phi0_params):
    """帶有不對稱內稟相位移的電流密度 (用於產生不對稱的Ic(Phi)圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    phi0_intrinsic = phi0_int_func(x_coords, W_val, **phi0_params)
    return J_real * np.exp(-1j * phi0_intrinsic) # J_eff = J_c * exp(-i * phi_0_int)

# 選定的電流密度分佈與數學表達式 (使用HTML格式，避免LaTeX渲染問題)
current_distributions = {
    "均勻 (Uniform)": {
        "profile": uniform_Jc(x_coords, W),
        "math": "J<sub>c</sub>(x) = J<sub>0</sub> (constant)"
    },
    "對稱邊緣高斯 (Symm. Edge Gaussian)": {
        "profile": symmetric_edge_gaussian_Jc(x_coords, W, peak_width_factor=0.08, edge_offset_factor=0.05),
        "math": "J<sub>c</sub>(x) = exp(-(x-x<sub>1</sub>)<sup>2</sup>/(2σ<sup>2</sup>)) + exp(-(x-x<sub>2</sub>)<sup>2</sup>/(2σ<sup>2</sup>))"
    },
    "不對稱線性 (僅正值) (Asymm. Linear)": {
        "profile": asymmetric_linear_Jc(x_coords, W),
        "math": "J<sub>c</sub>(x) = 0.7 + 0.5 × (x/(W/2))"
    },
    "單邊階梯 (One-side Step)": {
        "profile": one_side_step_Jc(x_coords, W, step_start_norm=-0.5, step_end_norm=-0.2),
        "math": "J<sub>c</sub>(x) = { J<sub>0</sub> if x<sub>1</sub> ≤ x ≤ x<sub>2</sub>; 0 otherwise }"
    },
    "不對稱多階梯 (僅正值) (Asymm. Multi-Steps)": {
        "profile": asymmetric_multistep_Jc(x_coords, W, x_split_norm=-0.1, h1=1.0, h2=0.4),
        "math": "J<sub>c</sub>(x) = { h<sub>1</sub> if x ≤ x<sub>s</sub>; h<sub>2</sub> if x > x<sub>s</sub> }"
    },
    "雙邊均勻邊緣 (Symm. Edge Steps)": {
        "profile": symmetric_edge_steps_Jc(x_coords, W, depth_norm=0.15, base_level=0.0),
        "math": "J<sub>c</sub>(x) = { J<sub>0</sub> if |x| > W/2 - d; 0 otherwise }"
    },
    "雙邊異形邊緣 (Bilateral Diff. Shapes)": {
        "profile": bilateral_opposite_shape_edge_Jc(x_coords, W, left_step_end_norm=-0.2, left_h=0.9, right_peak_center_norm=0.3, right_peak_h=1.0, right_peak_sigma_norm=0.1),
        "math": "J<sub>c</sub>(x) = { h<sub>L</sub> (left step); h<sub>R</sub> exp(-(x-x<sub>R</sub>)<sup>2</sup>/(2σ<sub>R</sub><sup>2</sup>)) (right Gaussian) }"
    },
    "均勻Jc+線性相位梯度 (僅正值) (Shifted Uniform)": {
        "profile": Jc_with_linear_phase_gradient(x_coords, W, lambda x, w: uniform_Jc(x,w), phase_k0_norm=0.5),
        "math": "J<sub>eff</sub>(x) = J<sub>0</sub> exp(i k<sub>0</sub> x)"
    },
    "新定義線性相位梯度 (New Linear Phase)": {
        "profile": Jc_with_linear_phase_gradient_new(x_coords, W, J0_amplitude, k0),
        "math": f"J<sub>eff</sub>(x) = {J0_amplitude} × exp(i × {k0} × x)<br>= {J0_amplitude} × (cos({k0}x) + i × sin({k0}x))"
    },
    "均勻Jc+不對稱相位 (僅正值) (Asymmetric Ic(Phi))": {
        "profile": Jc_with_asymmetric_phi0(x_coords, W, 
                                              lambda x, w: uniform_Jc(x,w), # 基底 Jc(x)
                                              asymmetric_phi0_profile,     # 不對稱 phi_0^{int}(x) 函數
                                              {'A': np.pi*0.8, 'x_d_norm': -0.25, 'sigma_norm': 0.2}), # phi_0^{int}(x) 的參數
        "math": "J<sub>eff</sub>(x) = J<sub>0</sub> exp(-i φ<sub>0</sub><sup>int</sup>(x)),<br>where φ<sub>0</sub><sup>int</sup>(x) = A exp(-(x-x<sub>d</sub>)<sup>2</sup>/(2σ<sup>2</sup>))"
    }
}

# 歸一化電流密度分佈
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    if np.iscomplexobj(Jc_profile_complex):
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9:
            current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val
    else: 
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9: 
             current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val 

# 計算 Ic(Phi) 圖樣
Ic_patterns = {}
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    Ic_vs_phi = []
    for phi_norm in phi_ext_over_phi0_scan:
        integrand = Jc_profile_complex * np.exp(1j * 2 * np.pi * phi_norm * (x_coords / W))
        dx = W / (N_points_x -1) if N_points_x > 1 else W 
        Ic_val = np.abs(np.sum(integrand) * dx) 
        Ic_vs_phi.append(Ic_val)

    Ic_patterns[name] = np.array(Ic_vs_phi)
    max_ic_val_pattern = np.max(Ic_patterns[name])
    if max_ic_val_pattern > 1e-9:
         Ic_patterns[name] /= max_ic_val_pattern

# 創建主要圖表
fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=False,
                    row_heights=[0.35, 0.35, 0.3],
                    subplot_titles=("電流密度分佈 J<sub>eff</sub>(x) (實部與虛部，已歸一化至|·|<sub>max</sub>=1)", 
                                    "對應的夫琅禾費圖樣 I<sub>c</sub>(Φ<sub>ext</sub>) (已歸一化至I<sub>c,peak</sub>=1)",
                                    "數學表達式"))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# 繪製電流密度分佈
for i, (name, data) in enumerate(current_distributions.items()):
    Jc_profile_complex = data["profile"]
    color = colors[i % len(colors)]
    
    if np.iscomplexobj(Jc_profile_complex):
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.real(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (實部)', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.imag(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (虛部)', legendgroup=name, 
                                 line=dict(color=color, dash='dot')),
                      row=1, col=1)
    else:
        fig.add_trace(go.Scatter(x=x_coords/W, y=Jc_profile_complex, mode='lines', 
                                 name=f'{name}', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)

    # 繪製 Ic(Phi) 圖樣
    fig.add_trace(go.Scatter(x=phi_ext_over_phi0_scan, y=Ic_patterns[name], mode='lines',
                             name=f'{name} - Ic(Φ)', legendgroup=name, 
                             showlegend=False, 
                             line=dict(color=color)),
                  row=2, col=1)

# 添加數學表達式文本
math_text_html = ""
for i, (name, data) in enumerate(current_distributions.items()):
    math_text_html += f"<b>{i+1}. {name}:</b><br>{data['math']}<br><br>"

fig.add_annotation(
    text=math_text_html,
    xref="paper", yref="paper",
    x=0.05, y=0.28,
    xanchor="left", yanchor="top",
    showarrow=False,
    font=dict(size=10),
    bgcolor="rgba(255,255,255,0.9)",
    bordercolor="rgba(0,0,0,0.3)",
    borderwidth=1
)

# 更新軸標籤和範圍
fig.update_xaxes(title_text="歸一化接面位置 x/W", row=1, col=1)
fig.update_yaxes(title_text="歸一化有效電流密度 J<sub>eff</sub>(x)", row=1, col=1, range=[-1.1, 1.1]) 

fig.update_xaxes(title_text="歸一化外部磁通量 (Φ<sub>ext</sub> / Φ<sub>0</sub>)", row=2, col=1, dtick=1)
fig.update_yaxes(title_text="歸一化臨界電流 (I<sub>c</sub> / I<sub>c,peak</sub>)", row=2, col=1, range=[-0.05, 1.1]) 

# 隱藏第三個子圖的軸
fig.update_xaxes(visible=False, row=3, col=1)
fig.update_yaxes(visible=False, row=3, col=1)

fig.update_layout(
    height=1800, 
    title_text="電流密度分佈及其對應的夫琅禾費干涉圖樣與數學表達式",
    title_x=0.5,
    legend_title_text="有效電流密度分佈類型",
    legend_tracegroupgap = 5,
    font=dict(family="Arial", size=12)
)

fig.show()

# 創建額外的 J_eff(x) 詳細圖表，展示新參數定義的函數
fig_jeff = go.Figure()

# 計算並繪製 J_eff(x) 在完整範圍內的行為
J_eff_values = J_eff_formula(x_values, J0_amplitude, k0)

fig_jeff.add_trace(go.Scatter(x=x_values, y=np.real(J_eff_values), 
                              mode='lines', name='實部 Re[J_eff(x)]',
                              line=dict(color='blue')))
fig_jeff.add_trace(go.Scatter(x=x_values, y=np.imag(J_eff_values), 
                              mode='lines', name='虛部 Im[J_eff(x)]',
                              line=dict(color='red', dash='dash')))
fig_jeff.add_trace(go.Scatter(x=x_values, y=np.abs(J_eff_values), 
                              mode='lines', name='幅度 |J_eff(x)|',
                              line=dict(color='green', width=2)))

fig_jeff.update_layout(
    title=f"J<sub>eff</sub>(x) = {J0_amplitude} × exp(i × {k0} × x) 的詳細行為",
    xaxis_title="x",
    yaxis_title="J<sub>eff</sub>(x)",
    height=600,
    legend=dict(x=0.7, y=0.95)
)

fig_jeff.show()

# 輸出參數信息
print(f"使用的參數:")
print(f"J0_amplitude = {J0_amplitude}")
print(f"k0 = {k0}")
print(f"x範圍: [{x_min}, {x_max}]")
print(f"波長 λ = 2π/k0 = {2*np.pi/k0:.2f}")
print(f"在範圍內包含 {(x_max-x_min)/(2*np.pi/k0):.1f} 個完整波長")

使用的參數:
J0_amplitude = 1.0
k0 = 1.0
x範圍: [0, 12.566370614359172]
波長 λ = 2π/k0 = 6.28
在範圍內包含 2.0 個完整波長


In [27]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.fft import fft, fftshift
from scipy.special import jv, yv # Bessel functions J_n and Y_n
from scipy.interpolate import lagrange # Lagrange polynomials

# --- 常數與參數設定 ---
Phi0 = 1.0       # 磁通量量子 (歸一化)
W = 1.0          # 接面寬度 (歸一化)
N_points_x = 256 # 接面空間取樣點數
x_coords = np.linspace(-W/2, W/2, N_points_x) # 接面寬度座標

# 掃描的外部磁通量範圍 (歸一化)
phi_ext_over_phi0_scan = np.linspace(-8, 8, 800) # 擴大範圍以觀察更多旁瓣

# --- Parameters ---
J0_amplitude = 1.0  # Amplitude of J0 (assuming J0 is real for simplicity)
k0 = 1.0            # Wavenumber
x_min = 0
x_max = 4 * np.pi   # Plot over a few wavelengths
num_points = 200    # Number of points to plot

# --- Generate x values ---
x_values = np.linspace(x_min, x_max, num_points)

# --- Calculate J_eff(x) ---
# J_eff(x) = J0 * exp(i * k0 * x)
# Using Euler's formula: exp(i * theta) = cos(theta) + i * sin(theta)
# So, J_eff(x) = J0 * (cos(k0 * x) + i * sin(k0 * x))
def J_eff_formula(x, J0, k0):
    """
    計算有效電流密度 J_eff(x) = J0 * exp(i * k0 * x)
    返回複數值
    """
    return J0 * np.exp(1j * k0 * x)

# --- 定義不同的電流密度分佈函數 Jc(x) ---
def uniform_Jc(x_coords, W_val):
    """均勻電流密度"""
    return np.ones_like(x_coords)

def symmetric_edge_gaussian_Jc(x_coords, W_val, peak_width_factor=0.1, edge_offset_factor=0.05):
    """對稱邊緣集中的電流密度 (兩個高斯峰)"""
    sigma = W_val * peak_width_factor
    peak1_center = -W_val/2 + W_val * edge_offset_factor + sigma 
    peak2_center = W_val/2 - W_val * edge_offset_factor - sigma
    return np.exp(-(x_coords - peak1_center)**2 / (2 * sigma**2)) + \
           np.exp(-(x_coords - peak2_center)**2 / (2 * sigma**2))

def asymmetric_linear_Jc(x_coords, W_val):
    """不對稱線性梯度電流密度 (僅正值)"""
    return 0.7 + 0.5 * (x_coords / (W_val/2)) 

def one_side_step_Jc(x_coords, W_val, step_start_norm=-0.5, step_end_norm=-0.2):
    """只有一邊的不對稱邊緣集中 (階梯)"""
    J = np.zeros_like(x_coords)
    start_x = step_start_norm * W_val
    end_x = step_end_norm * W_val
    actual_start = np.maximum(start_x, -W_val/2)
    actual_end = np.minimum(end_x, W_val/2)
    if actual_start < actual_end:
         J[(x_coords >= actual_start) & (x_coords <= actual_end)] = 1.0
    return J

def asymmetric_multistep_Jc(x_coords, W_val, x_split_norm=-0.1, h1=1.0, h2=0.4):
    """不對稱多階梯 (兩段不同高度, 僅正值)"""
    J = np.zeros_like(x_coords)
    x_split = x_split_norm * W_val
    J[x_coords <= x_split] = h1
    J[x_coords > x_split] = h2
    J[x_coords < -W_val/2] = 0
    J[x_coords > W_val/2] = 0
    return J

def symmetric_edge_steps_Jc(x_coords, W_val, depth_norm=0.2, base_level=0.0):
    """雙邊均勻邊緣集中 (對稱階梯)"""
    J = np.full_like(x_coords, base_level)
    edge_width = depth_norm * W_val
    J[(x_coords >= -W_val/2) & (x_coords <= -W_val/2 + edge_width)] = 1.0
    J[(x_coords >= W_val/2 - edge_width) & (x_coords <= W_val/2)] = 1.0
    return J

def bilateral_opposite_shape_edge_Jc(x_coords, W_val, 
                                     left_step_end_norm=-0.25, left_h=1.0,
                                     right_peak_center_norm=0.3, right_peak_h=0.8, right_peak_sigma_norm=0.07):
    """雙邊異形邊緣集中 (左階梯, 右高斯)"""
    J_left = np.zeros_like(x_coords)
    left_step_end = left_step_end_norm * W_val
    J_left[(x_coords >= -W_val/2) & (x_coords <= left_step_end)] = left_h 

    J_right = np.zeros_like(x_coords)
    right_peak_center = right_peak_center_norm * W_val
    right_sigma = right_peak_sigma_norm * W_val
    J_right = right_peak_h * np.exp(-(x_coords - right_peak_center)**2 / (2 * right_sigma**2)) 

    J_combined = np.zeros_like(x_coords)
    mask_left = (x_coords >= -W_val/2) & (x_coords <= left_step_end)
    mask_right_exclusive = (x_coords > left_step_end) 

    J_combined[mask_left] = J_left[mask_left]
    J_combined[mask_right_exclusive] = J_right[mask_right_exclusive]
    return J_combined

def Jc_with_linear_phase_gradient(x_coords, W_val, J_amplitude_func, phase_k0_norm):
    """帶有內稟線性相位梯度的電流密度 (用於產生位移的對稱圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    k_intrinsic = phase_k0_norm * np.pi 
    return J_real * np.exp(1j * k_intrinsic * (x_coords / (W/2)))

def Jc_with_linear_phase_gradient_new(x_coords, W_val, J0=1.0, k0_val=1.0):
    """使用新參數定義的線性相位梯度電流密度"""
    # 將 x_coords 從 [-W/2, W/2] 映射到 [x_min, x_max] 範圍
    x_mapped = (x_coords + W_val/2) / W_val * (x_max - x_min) + x_min
    return J_eff_formula(x_mapped, J0, k0_val)

def asymmetric_phi0_profile(x_coords, W_val, A=np.pi/2, x_d_norm=-0.2, sigma_norm=0.15):
    """一個不對稱的內稟相位移 phi_0^{int}(x) 範例 (例如，一側的相位突變)"""
    x_d = x_d_norm * W_val
    sigma = sigma_norm * W_val
    return A * np.exp(-(x_coords - x_d)**2 / (2 * sigma**2))

def Jc_with_asymmetric_phi0(x_coords, W_val, J_amplitude_func, phi0_int_func, phi0_params):
    """帶有不對稱內稟相位移的電流密度 (用於產生不對稱的Ic(Phi)圖樣)"""
    J_real = J_amplitude_func(x_coords, W_val)
    phi0_intrinsic = phi0_int_func(x_coords, W_val, **phi0_params)
    return J_real * np.exp(-1j * phi0_intrinsic) # J_eff = J_c * exp(-i * phi_0_int)

# 選定的電流密度分佈與數學表達式 (使用HTML格式，避免LaTeX渲染問題)
current_distributions = {
    "均勻 (Uniform)": {
        "profile": uniform_Jc(x_coords, W),
        "math": "J<sub>c</sub>(x) = J<sub>0</sub> (constant)"
    },
    "對稱邊緣高斯 (Symm. Edge Gaussian)": {
        "profile": symmetric_edge_gaussian_Jc(x_coords, W, peak_width_factor=0.08, edge_offset_factor=0.05),
        "math": "J<sub>c</sub>(x) = exp(-(x-x<sub>1</sub>)<sup>2</sup>/(2σ<sup>2</sup>)) + exp(-(x-x<sub>2</sub>)<sup>2</sup>/(2σ<sup>2</sup>))"
    },
    "不對稱線性 (僅正值) (Asymm. Linear)": {
        "profile": asymmetric_linear_Jc(x_coords, W),
        "math": "J<sub>c</sub>(x) = 0.7 + 0.5 × (x/(W/2))"
    },
    "單邊階梯 (One-side Step)": {
        "profile": one_side_step_Jc(x_coords, W, step_start_norm=-0.5, step_end_norm=-0.2),
        "math": "J<sub>c</sub>(x) = { J<sub>0</sub> if x<sub>1</sub> ≤ x ≤ x<sub>2</sub>; 0 otherwise }"
    },
    "不對稱多階梯 (僅正值) (Asymm. Multi-Steps)": {
        "profile": asymmetric_multistep_Jc(x_coords, W, x_split_norm=-0.1, h1=1.0, h2=0.4),
        "math": "J<sub>c</sub>(x) = { h<sub>1</sub> if x ≤ x<sub>s</sub>; h<sub>2</sub> if x > x<sub>s</sub> }"
    },
    "雙邊均勻邊緣 (Symm. Edge Steps)": {
        "profile": symmetric_edge_steps_Jc(x_coords, W, depth_norm=0.15, base_level=0.0),
        "math": "J<sub>c</sub>(x) = { J<sub>0</sub> if |x| > W/2 - d; 0 otherwise }"
    },
    "雙邊異形邊緣 (Bilateral Diff. Shapes)": {
        "profile": bilateral_opposite_shape_edge_Jc(x_coords, W, left_step_end_norm=-0.2, left_h=0.9, right_peak_center_norm=0.3, right_peak_h=1.0, right_peak_sigma_norm=0.1),
        "math": "J<sub>c</sub>(x) = { h<sub>L</sub> (left step); h<sub>R</sub> exp(-(x-x<sub>R</sub>)<sup>2</sup>/(2σ<sub>R</sub><sup>2</sup>)) (right Gaussian) }"
    },
    "均勻Jc+線性相位梯度 (僅正值) (Shifted Uniform)": {
        "profile": Jc_with_linear_phase_gradient(x_coords, W, lambda x, w: uniform_Jc(x,w), phase_k0_norm=0.5),
        "math": "J<sub>eff</sub>(x) = J<sub>0</sub> exp(i k<sub>0</sub> x)"
    },
    "新定義線性相位梯度 (New Linear Phase)": {
        "profile": Jc_with_linear_phase_gradient_new(x_coords, W, J0_amplitude, k0),
        "math": f"J<sub>eff</sub>(x) = {J0_amplitude} × exp(i × {k0} × x)<br>= {J0_amplitude} × (cos({k0}x) + i × sin({k0}x))"
    },
    "均勻Jc+不對稱相位 (僅正值) (Asymmetric Ic(Phi))": {
        "profile": Jc_with_asymmetric_phi0(x_coords, W, 
                                              lambda x, w: uniform_Jc(x,w), # 基底 Jc(x)
                                              asymmetric_phi0_profile,     # 不對稱 phi_0^{int}(x) 函數
                                              {'A': np.pi*0.8, 'x_d_norm': -0.25, 'sigma_norm': 0.2}), # phi_0^{int}(x) 的參數
        "math": "J<sub>eff</sub>(x) = J<sub>0</sub> exp(-i φ<sub>0</sub><sup>int</sup>(x)),<br>where φ<sub>0</sub><sup>int</sup>(x) = A exp(-(x-x<sub>d</sub>)<sup>2</sup>/(2σ<sup>2</sup>))"
    }
}

# 歸一化電流密度分佈
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    if np.iscomplexobj(Jc_profile_complex):
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9:
            current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val
    else: 
        max_abs_val = np.max(np.abs(Jc_profile_complex))
        if max_abs_val > 1e-9: 
             current_distributions[name]["profile"] = Jc_profile_complex / max_abs_val 

# 計算 Ic(Phi) 圖樣
Ic_patterns = {}
for name, data in current_distributions.items():
    Jc_profile_complex = data["profile"]
    Ic_vs_phi = []
    for phi_norm in phi_ext_over_phi0_scan:
        integrand = Jc_profile_complex * np.exp(1j * 2 * np.pi * phi_norm * (x_coords / W))
        dx = W / (N_points_x -1) if N_points_x > 1 else W 
        Ic_val = np.abs(np.sum(integrand) * dx) 
        Ic_vs_phi.append(Ic_val)

    Ic_patterns[name] = np.array(Ic_vs_phi)
    max_ic_val_pattern = np.max(Ic_patterns[name])
    if max_ic_val_pattern > 1e-9:
         Ic_patterns[name] /= max_ic_val_pattern

# 創建主要圖表
fig = make_subplots(rows=3, cols=1,
                    shared_xaxes=False,
                    row_heights=[0.35, 0.35, 0.3],
                    subplot_titles=("電流密度分佈 J<sub>eff</sub>(x) (實部與虛部，已歸一化至|·|<sub>max</sub>=1)", 
                                    "對應的夫琅禾費圖樣 I<sub>c</sub>(Φ<sub>ext</sub>) (已歸一化至I<sub>c,peak</sub>=1)",
                                    "數學表達式"))

colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728', '#9467bd', 
          '#8c564b', '#e377c2', '#7f7f7f', '#bcbd22', '#17becf']

# 繪製電流密度分佈
for i, (name, data) in enumerate(current_distributions.items()):
    Jc_profile_complex = data["profile"]
    color = colors[i % len(colors)]
    
    if np.iscomplexobj(Jc_profile_complex):
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.real(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (實部)', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)
        fig.add_trace(go.Scatter(x=x_coords/W, y=np.imag(Jc_profile_complex), mode='lines', 
                                 name=f'{name} (虛部)', legendgroup=name, 
                                 line=dict(color=color, dash='dot')),
                      row=1, col=1)
    else:
        fig.add_trace(go.Scatter(x=x_coords/W, y=Jc_profile_complex, mode='lines', 
                                 name=f'{name}', legendgroup=name, 
                                 line=dict(color=color)),
                      row=1, col=1)

    # 繪製 Ic(Phi) 圖樣
    fig.add_trace(go.Scatter(x=phi_ext_over_phi0_scan, y=Ic_patterns[name], mode='lines',
                             name=f'{name} - Ic(Φ)', legendgroup=name, 
                             showlegend=False, 
                             line=dict(color=color)),
                  row=2, col=1)

# 添加數學表達式文本
math_text_html = ""
for i, (name, data) in enumerate(current_distributions.items()):
    math_text_html += f"<b>{i+1}. {name}:</b><br>{data['math']}<br><br>"

fig.add_annotation(
    text=math_text_html,
    xref="paper", yref="paper",
    x=0.05, y=0.28,
    xanchor="left", yanchor="top",
    showarrow=False,
    font=dict(size=10),
    bgcolor="rgba(255,255,255,0.9)",
    bordercolor="rgba(0,0,0,0.3)",
    borderwidth=1
)

# 更新軸標籤和範圍
fig.update_xaxes(title_text="歸一化接面位置 x/W", row=1, col=1)
fig.update_yaxes(title_text="歸一化有效電流密度 J<sub>eff</sub>(x)", row=1, col=1, range=[-1.1, 1.1]) 

fig.update_xaxes(title_text="歸一化外部磁通量 (Φ<sub>ext</sub> / Φ<sub>0</sub>)", row=2, col=1, dtick=1)
fig.update_yaxes(title_text="歸一化臨界電流 (I<sub>c</sub> / I<sub>c,peak</sub>)", row=2, col=1, range=[-0.05, 1.1]) 

# 隱藏第三個子圖的軸
fig.update_xaxes(visible=False, row=3, col=1)
fig.update_yaxes(visible=False, row=3, col=1)

fig.update_layout(
    height=1800, 
    title_text="電流密度分佈及其對應的夫琅禾費干涉圖樣與數學表達式",
    title_x=0.5,
    legend_title_text="有效電流密度分佈類型",
    legend_tracegroupgap = 5,
    font=dict(family="Arial", size=12)
)

fig.show()

# 創建額外的 J_eff(x) 詳細圖表，展示新參數定義的函數
fig_jeff = go.Figure()

# 計算並繪製 J_eff(x) 在完整範圍內的行為
J_eff_values = J_eff_formula(x_values, J0_amplitude, k0)

fig_jeff.add_trace(go.Scatter(x=x_values, y=np.real(J_eff_values), 
                              mode='lines', name='實部 Re[J_eff(x)]',
                              line=dict(color='blue')))
fig_jeff.add_trace(go.Scatter(x=x_values, y=np.imag(J_eff_values), 
                              mode='lines', name='虛部 Im[J_eff(x)]',
                              line=dict(color='red', dash='dash')))
fig_jeff.add_trace(go.Scatter(x=x_values, y=np.abs(J_eff_values), 
                              mode='lines', name='幅度 |J_eff(x)|',
                              line=dict(color='green', width=2)))

fig_jeff.update_layout(
    title=f"J<sub>eff</sub>(x) = {J0_amplitude} × exp(i × {k0} × x) 的詳細行為",
    xaxis_title="x",
    yaxis_title="J<sub>eff</sub>(x)",
    height=600,
    legend=dict(x=0.7, y=0.95)
)

fig_jeff.show()

# 輸出參數信息
print(f"使用的參數:")
print(f"J0_amplitude = {J0_amplitude}")
print(f"k0 = {k0}")
print(f"x範圍: [{x_min}, {x_max}]")
print(f"波長 λ = 2π/k0 = {2*np.pi/k0:.2f}")
print(f"在範圍內包含 {(x_max-x_min)/(2*np.pi/k0):.1f} 個完整波長")

使用的參數:
J0_amplitude = 1.0
k0 = 1.0
x範圍: [0, 12.566370614359172]
波長 λ = 2π/k0 = 6.28
在範圍內包含 2.0 個完整波長
